In [2]:
# Dependencies
import requests as req
import json
import tweepy
import pandas as pd
import time as t
import matplotlib.pyplot as plt
from config import (consumer_key, consumer_secret, access_token, access_token_secret)
from datetime import datetime, date, time
import numpy as np

# Initialize sentiment analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#Setup Tweepy API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

#Saving real time stock data in  file
stockHistory = 'stockHistory.csv'
# Base URL for GET requests to retrieve stock data
url = "https://api.iextrading.com/1.0/stock/"
#Opening file
file=open(stockHistory,'w')

#Stock ticker for collecting data
stocks = ['AAPL', 'V', 'AMZN', 'WMT', 'NFLX', 'NKE', 'M']
# Specify the companies
target_terms = ["@Apple", "@Visa", "@amazon", "@Walmart", "@netflix", "@Nike", "@Macys"]

# Loop through (can change the range if we need to)
def getSentimentAnalysis(handler):
    # Loop through all companies
    
   # Make a dictionary to hold each type of sentiment
        company_sentiment = {
            "compound_list": [], 
            "positive_list": [], 
            "negative_list": [], 
            "neutral_list": []
        }
        public_tweets = api.search(handler, count=100, result_type="recent")

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]

            # Save all info to the corresponding lists
            company_sentiment["compound_list"].append(compound)
            company_sentiment["positive_list"].append(pos)
            company_sentiment["neutral_list"].append(neu)
            company_sentiment["negative_list"].append(neg)
        list_of_sentiment = [np.mean(company_sentiment["compound_list"]),np.mean(company_sentiment["positive_list"]),
                np.mean(company_sentiment["neutral_list"]),np.mean(company_sentiment["negative_list"])]
        return list_of_sentiment
print(str(datetime.now().time()))

13:09:49.181136


In [ ]:
stock_market_open_time = time(hour=8, minute=30, second=0, microsecond=0)
stock_market_close_time = time(hour=23, minute=0, second=0, microsecond=0)

file.write(" Counter  ,"+"Corporation          ,"+"Time       ,"+"Stock Ticker       ,"+"Stock Price       ,"+
           "Compound         ,"+"Positive         ,"+"Neutral        ,"+"Negative        \n")
def stockPriceCollectionWithSentiment():
    counter = 0;
    while((datetime.now().time() > stock_market_open_time) & (datetime.now().time() < stock_market_close_time) & counter < 50):
        counter = counter+1
        time_var = t.mktime(datetime.now().timetuple()) * 1000
        for stock in stocks:
            print(url+stock+"/quote")
            response = req.get(url+stock+"/quote")
            print("Counter::"+str(counter))
            print(response.json())
            sentiment_anal = getSentimentAnalysis(target_terms[stocks.index(stock)])
            print("Compound::",sentiment_anal[0])
            print("Positive::",sentiment_anal[1])
            print("Neutral::",sentiment_anal[2])
            print("Negative::",sentiment_anal[3])
            print(f"{counter} ,{response.json()['companyName']}          ,{time_var}        ,{response.json()['symbol']}     ,{response.json()['latestPrice']}     ," 
                       +f"{sentiment_anal[0]}      ,{sentiment_anal[1]}      ,{sentiment_anal[2]}      ,{sentiment_anal[3]}\n")
            file.write(f"{counter} ,{response.json()['companyName']}          ,{time_var}        ,{response.json()['symbol']}     ,{response.json()['latestPrice']}     ," 
                       +f"{sentiment_anal[0]}      ,{sentiment_anal[1]}      ,{sentiment_anal[2]}      ,{sentiment_anal[3]}\n")
        t.sleep(60)
try:
    stockPriceCollectionWithSentiment()
except KeyboardInterrupt:
    print('\n\nKeyboard exception received. Exiting.')
    file.close()
    exit()
file.close();



https://api.iextrading.com/1.0/stock/AAPL/quote
Counter::1
{'symbol': 'AAPL', 'companyName': 'Apple Inc.', 'primaryExchange': 'Nasdaq Global Select', 'sector': 'Technology', 'calculationPrice': 'close', 'open': 169.96, 'openTime': 1512138600451, 'close': 171.05, 'closeTime': 1512162000303, 'latestPrice': 171.05, 'latestSource': 'Close', 'latestTime': 'December 1, 2017', 'latestUpdate': 1512162000303, 'latestVolume': 39731614, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 171.11, 'delayedPriceTime': 1512165595000, 'previousClose': 171.85, 'change': -0.8, 'changePercent': -0.00466, 'iexMarketPercent': None, 'iexVolume': None, 'avgTotalVolume': 29859170, 'iexBidPrice': None, 'iexBidSize': None, 'iexAskPrice': None, 'iexAskSize': None, 'marketCap': 878224067600, 'peRatio': 19.44, 'week52High': 176.24, 'week52Low': 108.25, 'ytdChange': 0.47266465777012484}
Compound:: 0.023724
Positive:: 0.0369
Neutral:: 0.94066
Negative:: 0.02243
1 ,Apple Inc.   